<a href="https://colab.research.google.com/github/happkiller1010101/test_ai/blob/testdev/Copy_of_gpt_llm_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Describe your model -> fine-tuned LLaMA 2
By Matt Shumer (https://twitter.com/mattshumer_)

The goal of this notebook is to experiment with a new way to make it very easy to build a task-specific model for your use-case.

First, use the best GPU available (go to Runtime -> change runtime type)

To create your model, just go to the first code cell, and describe the model you want to build in the prompt. Be descriptive and clear.

Select a temperature (high=creative, low=precise), and the number of training examples to generate to train the model. From there, just run all the cells.

You can change the model you want to fine-tune by changing `model_name` in the `Define Hyperparameters` cell.

#Data generation step

Write your prompt here. Make it as descriptive as possible!

Then, choose the temperature (between 0 and 1) to use when generating data. Lower values are great for precise tasks, like writing code, whereas larger values are better for creative tasks, like writing stories.

Finally, choose how many examples you want to generate. The more you generate, a) the longer it takes and b) the more expensive data generation will be. But generally, more examples will lead to a higher-quality model. 100 is usually the minimum to start.

In [ ]:
prompt = "A model that takes in a puzzle-like reasoning-heavy question in English, and responds with a well-reasoned, step-by-step thought out response in Spanish."
temperature = .4
number_of_examples = 100

Run this to generate the dataset.

In [ ]:
!pip install openai

In [ ]:
import os
import openai
import random

openai.api_key = "YOUR KEY HERE"

def generate_example(prompt, prev_examples, temperature=.5):
    messages=[
        {
            "role": "system",
            "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 10:
            prev_examples = random.sample(prev_examples, 10)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        temperature=temperature,
        max_tokens=1354,
    )

    return response.choices[0].message['content']

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)

print(prev_examples)

We also need to generate a system message.

In [ ]:
def generate_system_message(prompt):

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
          {
            "role": "system",
            "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
          },
          {
              "role": "user",
              "content": prompt.strip(),
          }
        ],
        temperature=temperature,
        max_tokens=500,
    )

    return response.choices[0].message['content']

system_message = generate_system_message(prompt)

print(f'The system message is: `{system_message}`. Feel free to re-run this cell if you want a better result.')

Now let's put our examples into a dataframe and turn them into a final pair of datasets.

In [ ]:
import pandas as pd

# Load the dataset from a CSV file into a pandas DataFrame
prev_examples = pd.read_csv('test_case_dataset.csv')
print(prev_examples)

# Initialize lists to store prompts and responses
User_Story = []
Scenario = []
Given = []
When = []
And = []
Then = []

# Iterate over the rows of the DataFrame
for index, row in prev_examples.iterrows():
    try:
        # Assuming 'User_Story', 'Scenario', 'Given', 'When', 'And', 'Then' are column names in your CSV
        User_Story.append(row['User_Story'].strip())
        Scenario.append(row['Scenario'].strip())
        Given.append(row['Given'].strip())
        When.append(row['When'].strip())
        And.append(row['And'].strip())
        Then.append(row['Then'].strip())
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        pass

# Create a DataFrame from the lists
df = pd.DataFrame({
    'User_Story': User_Story,
    'Scenario': Scenario,
    'Given': Given,
    'When': When,
    'And': And,
    'Then': Then
})

# Remove duplicates
df_result = df.drop_duplicates()

print('There are ' + str(len(df_result)) + ' successfully-generated examples. Here are the first few:')

# Display the first few rows
df.head()


                                           User_Story  \
0   As a user, I want to manage products in my sho...   
1   As a user, I want to manage products in my sho...   
2   As a user, I want to search for products on th...   
3   As a user, I want to update the quantity of it...   
4   As a user, I want to apply discount coupons on...   
5   As a user, I want to view my order history to ...   
6   As a user, I want to filter products by catego...   
7   As a user, I want to update the quantity of it...   
8   As a user, I want to apply discount coupons on...   
9   As a user, I want to apply discount coupons on...   
10  As a user, I want to log in to the application...   
11  As a user, I want to log in to the application...   
12  As a user, I want to log in to the application...   
13  As a user, I want to log in to the application...   
14  As a user, I want to update my account details...   
15  As a user, I want to sort products by price to...   
16  As a user, I want to sort p

,User_Story,Scenario,Given,When,And,Then
0,"As a user, I want to manage products in my sho...",Validate that user can add one item to cart,the user launches and logs into the e-commerce...,the user navigates to the items page,the user selects and clicks on a <item> and th...,the user should navigate to the Shopping Cart ...
1,"As a user, I want to manage products in my sho...",Validate that user can remove one item from cart,the user launches and logs into the e-commerce...,the user navigates to the Shopping Cart page,the user finds <item> in the shopping cart and...,the user should validate <item> in Shopping Ca...
2,"As a user, I want to search for products on th...",Validate that user can search for a product by...,the user launches and logs into the e-commerce...,the user enters <product name> in the search bar,the user clicks the search button and views th...,the user should see a list of products matchin...
3,"As a user, I want to update the quantity of it...",Validate that user can increase the quantity o...,the user launches and logs into the e-commerce...,the user navigates to the Shopping Cart page,the user increases the quantity of <item> by c...,the user should see the updated quantity of <i...
4,"As a user, I want to apply discount coupons on...",Validate that user can apply a discount coupon...,the user launches and logs into the e-commerce...,the user navigates to the Checkout page,the user enters the discount coupon code in th...,the total price should be updated to reflect t...


Split into train and test sets.

In [ ]:
# Split the data into train and test sets, with 90% in the train set
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# Save the dataframes to .jsonl files
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)

# Install necessary libraries

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
!pip install pyarrow datasets

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Define Hyperparameters

In [ ]:
model_name = "NousResearch/llama-2-7b-chat-hf" # use this if you have access to the official LLaMA 2 model "meta-llama/Llama-2-7b-chat-hf", though keep in mind you'll need to pass a Hugging Face key argument
dataset_name = "/content/train.jsonl"
new_model = "llama-2-7b-custom"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

#Load Datasets and Train

In [ ]:
# Load datasets
train_dataset = load_dataset('json', data_files='train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='test.jsonl', split="train")

#Preprocess dataset
train_dataset_mapped = train_dataset.map(lambda examples: {
    'text': [
        f"[INST]\n\n{User_Story} [/INST]\n\n"
        f"Scenario: {Scenario}\n"
        f"Given: {Given}\n"
        f"When: {When}\n"
        f"And: {And}\n"
        f"Then: {Then}\n"
        for User_Story, Scenario, Given, When, And, Then in zip(
            examples['User_Story'], examples['Scenario'], examples['Given'],
            examples['When'], examples['And'], examples['Then']
        )
    ]}, batched=True)

valid_dataset_mapped = valid_dataset.map(lambda examples: {
    'text': [
        f"[INST]\n\n{User_Story} [/INST]\n\n"
        f"Scenario: {Scenario}\n"
        f"Given: {Given}\n"
        f"When: {When}\n"
        f"And: {And}\n"
        f"Then: {Then}\n"
        for User_Story, Scenario, Given, When, And, Then in zip(
            examples['User_Story'], examples['Scenario'], examples['Given'],
            examples['When'], examples['And'], examples['Then']
        )
    ]}, batched=True)



compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5  # Evaluate every 20 steps
)
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)
trainer.train()
trainer.model.save_pretrained(new_model)

# Cell 4: Test the model
# Cell 4: Test the model
logging.set_verbosity(logging.CRITICAL)

# Adjust the test User_Story to explicitly request the output structure
User_Story = f"[INST]\n\nAs a user, I want to manage products in my shopping basket on an e-commerce website to make future purchases. [/INST] \n\nPlease provide the following:\n\nScenario:\nGiven:\nWhen:\nAnd:\nThen:"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

result = pipe(User_Story)

# Print the generated text, expecting the model to fill in the structured fields
print(result[0]['generated_text'])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
5,2.621100,2.220157


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


[INST]

As a user, I want to manage products in my shopping basket on an e-commerce website to make future purchases. [/INST] 

Please provide the following:

Scenario:
Given:
When:
And:
Then:

Given:
* I am on an e-commerce website and browsing products.
* I add several products to my shopping basket.

When:
* I click on the shopping basket icon in the top right corner of the page.
* I view the products in my shopping basket.

And:
* I can view the quantity of each product in my shopping basket.
* I can remove products from my shopping basket.
* I can update the quantity of a product in my shopping basket.

Then:
* I am able to proceed to checkout and complete my purchase.

Scenario:



#Run Inference

In [ ]:
# from transformers import pipeline

# prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\nWrite a function that reverses a string. [/INST]" # replace the command here with something relevant to your task
# num_new_tokens = 100  # change to the number of new tokens you want to generate

# # Count the number of tokens in the prompt
# num_prompt_tokens = len(tokenizer(prompt)['input_ids'])

# # Calculate the maximum length for the generation
# max_length = num_prompt_tokens + num_new_tokens

# gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
# result = gen(prompt)
# print(result[0]['generated_text'].replace(prompt, ''))


# Adjust the test User_Story to explicitly request the output structure
User_Story = (
    "[INST]As a user, I want to manage products in my shopping basket on an e-commerce website to make future purchases.[/INST]"
    "\n\nPlease provide the following:\n\n"
    "Scenario: Validate that the user can log in successfully.\n"
    "Given: The user launches and logs into the e-commerce application with <username> and <password>.\n"
    "When: The user navigates to the account page.\n"
    "And: The user accesses the account dashboard.\n"
    "Then: The user should be able to view account details.\n"
)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(User_Story)

# # Print the generated text, expecting the model to fill in the structured fields
# print(result[0]['generated_text'])


# Test with the generated text

print(result[0]['generated_text'])


[INST]As a user, I want to manage products in my shopping basket on an e-commerce website to make future purchases.[/INST]

Please provide the following:

Scenario: Validate that the user can log in successfully.
Given: The user launches and logs into the e-commerce application with <username> and <password>.
When: The user navigates to the account page.
And: The user accesses the account dashboard.
Then: The user should be able to view account details.

Scenario: Validate that the user can view their shopping basket.
Given: The user navigates to the shopping basket page.
When: The user clicks on the shopping basket link.
And: The user views the shopping basket contents.
Then: The user should be able to view the products in their shopping basket.

Scenario: Validate that the user


#Merge the model and store in Google Drive

In [ ]:
text = result[0]['generated_text']

text2 = text.replace("Please provide the following:", "")
text3 = text2.replace("Scenario: Validate that the user can log in successfully.", "")
text4 = text3.replace("Given: The user launches and logs into the e-commerce application with <username> and <password>.", "")
text5 = text4.replace("When: The user navigates to the account page.", "")
text6 = text5.replace("And: The user accesses the account dashboard.", "")
text7 = text6.replace("Then: The user should be able to view account details.", "")

print(text7)

[INST]As a user, I want to search for products on the e-commerce website to find what I need.[/INST]









Scenario: Validate that the user can search for products.
Given: The user launches the e-commerce application.
When: The user navigates to the product search page.
And: The user enters a search query.
Then: The user should be able to view search results.

Scenario: Validate that the user can view product details.
Given: The user navigates to


In [ ]:
# # Merge and save the fine-tuned model
# from google.colab import drive
# drive.mount('/content/drive')

model_path = "/content/drive/llama-2-7b-custom"  # change to your preferred path

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Save the merged model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 

# Load a fine-tuned model from Drive and run inference

In [ ]:
from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer

drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/llama-2-7b-custom"  # change to the path where your model is saved

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
from transformers import pipeline

prompt = "What is 2 + 2?"  # change to your desired prompt
gen = pipeline('text-generation', model=model, tokenizer=tokenizer)
result = gen(prompt)
print(result[0]['generated_text'])